In [23]:
import json
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import argparse
import os
import word2vec
import visualizer as VS
import eval_utils as EU
import numpy as np
import pandas as pd
from gensim.models import Word2Vec, KeyedVectors
import sys

%reload_ext autoreload
%autoreload 2

In [24]:
embedding_storage = {
    "node2vec": '../node2vec/emb/',
    "ProNE": '../ProNE/emb/'
}

with open("../embedding_config.json", "r") as jsonfile:
    embeddding_config = json.load(jsonfile)
num_bins = embeddding_config["num_bins"]
test_size = embeddding_config["test_size"]
task = "genes"
method = "node2vec"

In [25]:
with open("../data/data_config.txt", "r") as jsonfile:
    data_config = json.load(jsonfile)
with open("../data/strategies/" + task + ".txt", "r") as jsonfile:
    strategies = json.load(jsonfile)
config = data_config[task]
location = config["location"]
target_file = config["target_file"]
location_processed = config["location_processed"]
target_column = config["target_column"]

In [26]:
# Load data
trimmed_table = pd.read_csv(os.path.join("../", location_processed), sep=',', encoding='latin')
full_table = pd.read_csv(os.path.join("../", location + target_file), sep=',', encoding='latin')

Y = full_table[target_column]
if task in ["kraken", "financial", "genes"]:
    Y = pd.Categorical(Y).codes

# Set embeddings that are to be evaluated
all_embeddings_path = EU.all_files_in_path(embedding_storage[method], task)

In [27]:
path = all_embeddings_path[0]

In [ ]:
model = KeyedVectors.load_word2vec_format(path)
table_name = path.split("/")[-1][:-4]
model_dict_path = "../graph/{}/{}.dict".format(task, table_name)
print(model_dict_path)

# Obtain textified & quantized data
training_loss = []
testing_loss = []
for i in range(50, 100, 20):
    df_textified = EU.textify_df(
        trimmed_table, strategies, location_processed)
    x_vec = EU.vectorize_df(
        df_textified, model, model.vocab,
        model_dict=model_dict_path, model_type=method
    )

    model_2dim = EU.get_PCA_for_embedding(model, ndim=i)
    x_vec_2dim = EU.vectorize_df(
        df_textified, model_2dim, model.vocab,
        model_dict=model_dict_path, model_type=method
    )
    tests = train_test_split(x_vec_2dim, Y, test_size=test_size, random_state=10)
    train_loss, test_loss = EU.classification_task_nn(*tests)
    training_loss.append(train_loss)
    testing_loss.append(test_loss)

../graph/genes/genes.dict
Epoch 1/500
22/22 [==============================] - 0s 2ms/step - loss: 2.6943 - accuracy: 0.0926
Epoch 2/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4444 - accuracy: 0.4410
Epoch 3/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4085 - accuracy: 0.4174
Epoch 4/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3571 - accuracy: 0.4656
Epoch 5/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3680 - accuracy: 0.4527
Epoch 6/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4150 - accuracy: 0.4041
Epoch 7/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3927 - accuracy: 0.4260
Epoch 8/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3767 - accuracy: 0.4419
Epoch 9/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3992 - accuracy: 0.4188
Epoch 10/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3887 - 

22/22 [==============================] - 0s 2ms/step - loss: 2.3930 - accuracy: 0.4236
Epoch 83/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3735 - accuracy: 0.4431
Epoch 84/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3514 - accuracy: 0.4651
Epoch 85/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3962 - accuracy: 0.4203
Epoch 86/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3932 - accuracy: 0.4234
Epoch 87/500
22/22 [==============================] - 0s 3ms/step - loss: 2.4030 - accuracy: 0.4135
Epoch 88/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3977 - accuracy: 0.4188
Epoch 89/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3536 - accuracy: 0.4629
Epoch 90/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3715 - accuracy: 0.4450
Epoch 91/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3855 - accuracy: 0.4310
Epoch 92/500


22/22 [==============================] - 0s 2ms/step - loss: 2.3718 - accuracy: 0.4447
Epoch 164/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4036 - accuracy: 0.4129
Epoch 165/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3641 - accuracy: 0.4524
Epoch 166/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3732 - accuracy: 0.4433
Epoch 167/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3902 - accuracy: 0.4263
Epoch 168/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4047 - accuracy: 0.4118
Epoch 169/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4001 - accuracy: 0.4164
Epoch 170/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3819 - accuracy: 0.4346
Epoch 171/500
22/22 [==============================] - 0s 2ms/step - loss: 2.4127 - accuracy: 0.4038
Epoch 172/500
22/22 [==============================] - 0s 2ms/step - loss: 2.3980 - accuracy: 0.4186
Epoc

In [96]:
import logging
tf.get_logger().setLevel(logging.ERROR)